In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def spirala(points_per_spiral=200, num_spirals=2, noise=0.01):
    X = []
    y = []
    for spiral in range(num_spirals):
        theta = np.linspace(0, 4 * np.pi, points_per_spiral)
        radius = np.linspace(0, 1, points_per_spiral)
        x1 = radius * np.cos(theta + spiral * np.pi) + noise * np.random.randn(points_per_spiral)
        x2 = radius * np.sin(theta + spiral * np.pi) + noise * np.random.randn(points_per_spiral)
        X.extend(np.c_[x1, x2])
        y.extend([spiral] * points_per_spiral)
    return np.array(X), np.array(y)

# יצירת הנתונים
X_spiral, y_spiral = spirala(points_per_spiral=200, noise=0.01)

In [ ]:
# use matplotlib to visualize the data
plt.scatter(X_spiral[:, 0], X_spiral[:, 1], c=y_spiral, cmap='coolwarm', edgecolors='k')
plt.title("Spiral Data Visualization")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

In [ ]:
# חלוקת הנתונים לאימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(X_spiral, y_spiral, test_size=0.2, random_state=42)

# המרת הנתונים לטנסורים
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
# הגדרת המודל
class Spiraling(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# יצירת אינסטנס של המודל
model = Spiraling()

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
)

In [ ]:
# הגדרת פונקציית הפסד ואופטימיזר
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# אימון המודל
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # תחזיות ואיבוד
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # חישוב גרדיאנטים ועדכון משקלים
    loss.backward()
    optimizer.step()

    # הדפסת איבוד כל 100 אפוקים
    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
            test_predictions = (torch.sigmoid(test_outputs) > 0.5).float()
            accuracy = (test_predictions == y_test_tensor).float().mean()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Test Accuracy: {accuracy.item()}")
        model.train()

# הערכת המודל הסופי
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_predictions = (torch.sigmoid(test_outputs) > 0.5).float()
    accuracy = (test_predictions == y_test_tensor).float().mean()

print(f"Final Test Accuracy: {accuracy.item() * 100:.2f}%")

In [ ]:
%run PlotBoudary.ipynb

In [ ]:
# הצגת תוצאת המודל על הספירלות
plot_boundary(X_spiral, y_spiral, model)